# Reading database into csv

In [17]:
import sqlite3
conn = sqlite3.connect('final.db')
cur = conn.cursor()

In [18]:
# Methods
def fetch_table_names(cur: sqlite3.Cursor) -> [str]:
    '''
    Retrieves all tables in the database
    '''
    
    cur.execute('SELECT name FROM sqlite_master WHERE type = \'table\';')
    return [x[0] for x in cur.fetchall()]

def fetch_column_names(cur: sqlite3.Cursor, table: str) -> [str]:
    '''
    Retrieves all columns in a table
    '''
    
    cur.execute('PRAGMA table_info(' + table + ');')
    return [x[1] for x in cur.fetchall()]

def fetch_col_values(cur: sqlite3.Cursor, table: str, col: str) -> []:
    '''
    Returns all values for a specific column
    '''
    
    cur.execute('SELECT ' + col + ' FROM ' + table + ';')
    return [x[0] for x in cur.fetchall()]

In [19]:
fetch_table_names(cur)

['SCHOOL_TRAIN', 'SCHOOL_TEST', 'POKEMON_TRAIN', 'POKEMON_TEST']

In [20]:
names = fetch_column_names(cur, 'POKEMON_TRAIN')
names
names2 = fetch_column_names(cur, 'POKEMON_TEST')
names2

['rowid',
 'unique_id',
 'types',
 'abilities',
 'base_happiness',
 'height_m',
 'weight_kg',
 'poke_stats']

In [21]:
import pandas as pd
import numpy as np
train = pd.DataFrame()
test = pd.DataFrame()

In [22]:
for name in names:
    train[name] = fetch_col_values(cur, 'POKEMON_TRAIN', name)

In [23]:
for name in names2:
    test[name] = fetch_col_values(cur, 'POKEMON_TRAIN', name)

In [24]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [25]:
train.shape,test.shape

((520, 9), (520, 8))

In [26]:
cleaner = pd.concat([train.copy(), test.copy()], ignore_index=True)

C:\Users\Loopy\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


# Cleaning

In [27]:
cleaner.head(10)

,abilities,base_happiness,height_m,is_legendary,poke_stats,rowid,types,unique_id,weight_kg
0,"['Guts', 'Sheer Force', 'Iron Fist']",70.0,0.6,0,"nan, 80.0, 55.0, 25.0, 35.0, 35.0",1,"fighting, nan",665,12.5
1,"['Bulletproof', 'Soundproof', 'Overcoat']",70.0,0.6,0,"nan, nan, 65.0, 45.0, 45.0, 45.0",2,"dragon, nan",288,29.7
2,"['Corrosion', 'Oblivious']",70.0,1.2,0,"nan, 64.0, 60.0, 111.0, 60.0, 117.0",3,"poison, fire",349,22.2
3,"['Rock Head', 'Lightningrod', 'Battle Armor']",70.0,0.4,0,"nan, 50.0, 95.0, 40.0, 50.0, 35.0",4,"ground, nan",260,6.5
4,"['Torrent', 'Sheer Force']",70.0,0.6,0,"nan, 65.0, 64.0, 44.0, 48.0, 43.0",5,"water, nan",536,9.5
5,['Turboblaze'],0.0,3.2,1,"nan, 120.0, 100.0, 150.0, 120.0, 90.0",6,"dragon, fire",293,330.0
6,['Levitate'],70.0,1.1,0,"nan, 70.0, 50.0, 50.0, 50.0, 70.0",7,"ground, dragon",181,15.3
7,"['Poison Point', 'Rivalry', 'Sheer Force']",70.0,1.3,0,"nan, 92.0, 87.0, 75.0, 85.0, 76.0",8,"poison, ground",554,60.0
8,['Battery'],70.0,0.5,0,"nan, 82.0, 95.0, 55.0, 75.0, 36.0",9,"bug, electric",507,10.5
9,['Beast Boost'],0.0,1.2,1,"109, 53.0, 47.0, 127.0, 131.0, 103.0",10,"rock, poison",140,55.5


### Get unique pokemon types in this df

In [28]:
unq = set()
main = []
for row in cleaner['types']:
    temp = row.split(', ')
    main.append(temp)
    for typ in temp:
        unq.add(typ)
print(unq)

for typ in unq:
    cleaner[typ] = 0

{'rock', 'dragon', 'flying', 'normal', 'psychic', 'fighting', 'dark', 'steel', 'water', 'bug', 'fairy', 'ground', 'poison', 'fire', 'grass', 'ghost', 'ice', 'nan', 'electric'}


### Fills in pokemon types in df
1 if pokemon is of that type, 0 else

This takes a good while to finish

In [29]:
for typ in unq:
    i = 0
    for var in cleaner[typ]:
        first,second = main[i][0],main[i][1]
        if (first == typ) or (second == typ):
            cleaner[typ][i] = 1
        i += 1

C:\Users\Loopy\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [30]:
cleaner.head()

,abilities,base_happiness,height_m,is_legendary,poke_stats,rowid,types,unique_id,weight_kg,rock,...,bug,fairy,ground,poison,fire,grass,ghost,ice,nan,electric
0,"['Guts', 'Sheer Force', 'Iron Fist']",70.0,0.6,0,"nan, 80.0, 55.0, 25.0, 35.0, 35.0",1,"fighting, nan",665,12.5,0,...,0,0,0,0,0,0,0,0,1,0
1,"['Bulletproof', 'Soundproof', 'Overcoat']",70.0,0.6,0,"nan, nan, 65.0, 45.0, 45.0, 45.0",2,"dragon, nan",288,29.7,0,...,0,0,0,0,0,0,0,0,1,0
2,"['Corrosion', 'Oblivious']",70.0,1.2,0,"nan, 64.0, 60.0, 111.0, 60.0, 117.0",3,"poison, fire",349,22.2,0,...,0,0,0,1,1,0,0,0,0,0
3,"['Rock Head', 'Lightningrod', 'Battle Armor']",70.0,0.4,0,"nan, 50.0, 95.0, 40.0, 50.0, 35.0",4,"ground, nan",260,6.5,0,...,0,0,1,0,0,0,0,0,1,0
4,"['Torrent', 'Sheer Force']",70.0,0.6,0,"nan, 65.0, 64.0, 44.0, 48.0, 43.0",5,"water, nan",536,9.5,0,...,0,0,0,0,0,0,0,0,1,0


In [31]:
import re

In [32]:
i = 0
for val in cleaner['abilities']:
    t = val
    t = re.sub('[\'\[\]]', '', t)
    t = t.split(', ')
    abSum = len(t)
    cleaner['abilities'][i] = abSum
    i += 1

C:\Users\Loopy\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [33]:
cleaner.head()

,abilities,base_happiness,height_m,is_legendary,poke_stats,rowid,types,unique_id,weight_kg,rock,...,bug,fairy,ground,poison,fire,grass,ghost,ice,nan,electric
0,3,70.0,0.6,0,"nan, 80.0, 55.0, 25.0, 35.0, 35.0",1,"fighting, nan",665,12.5,0,...,0,0,0,0,0,0,0,0,1,0
1,3,70.0,0.6,0,"nan, nan, 65.0, 45.0, 45.0, 45.0",2,"dragon, nan",288,29.7,0,...,0,0,0,0,0,0,0,0,1,0
2,2,70.0,1.2,0,"nan, 64.0, 60.0, 111.0, 60.0, 117.0",3,"poison, fire",349,22.2,0,...,0,0,0,1,1,0,0,0,0,0
3,3,70.0,0.4,0,"nan, 50.0, 95.0, 40.0, 50.0, 35.0",4,"ground, nan",260,6.5,0,...,0,0,1,0,0,0,0,0,1,0
4,2,70.0,0.6,0,"nan, 65.0, 64.0, 44.0, 48.0, 43.0",5,"water, nan",536,9.5,0,...,0,0,0,0,0,0,0,0,1,0


# Split poke_stats into seperate stat columns

In [34]:
poke_stats = cleaner['poke_stats']
stat_type = ['HP', 'Atk', 'Def', 'SpA', 'SpD', 'Spe']

In [36]:
stat_list = []
for i in range(0,len(cleaner['poke_stats'])):
    stat_list.append(poke_stats.iloc[i].split(', '))

In [37]:
stats = pd.DataFrame(stat_list, columns=stat_type)

In [38]:
stats.head()

,HP,Atk,Def,SpA,SpD,Spe
0,nan,80.0,55.0,25.0,35.0,35.0
1,nan,nan,65.0,45.0,45.0,45.0
2,nan,64.0,60.0,111.0,60.0,117.0
3,nan,50.0,95.0,40.0,50.0,35.0
4,nan,65.0,64.0,44.0,48.0,43.0


In [39]:
def combine_dfs(df1, df2):
    #adds df2 columns to the end of df1
    cols = df2.columns
    for col_names in cols:
        df1[col_names] = df2[col_names]
    return df1

In [40]:
cleaner = combine_dfs(cleaner, stats)

In [41]:
cleaner.head(3)

,abilities,base_happiness,height_m,is_legendary,poke_stats,rowid,types,unique_id,weight_kg,rock,...,ghost,ice,nan,electric,HP,Atk,Def,SpA,SpD,Spe
0,3,70.0,0.6,0,"nan, 80.0, 55.0, 25.0, 35.0, 35.0",1,"fighting, nan",665,12.5,0,...,0,0,1,0,nan,80.0,55.0,25.0,35.0,35.0
1,3,70.0,0.6,0,"nan, nan, 65.0, 45.0, 45.0, 45.0",2,"dragon, nan",288,29.7,0,...,0,0,1,0,nan,nan,65.0,45.0,45.0,45.0
2,2,70.0,1.2,0,"nan, 64.0, 60.0, 111.0, 60.0, 117.0",3,"poison, fire",349,22.2,0,...,0,0,0,0,nan,64.0,60.0,111.0,60.0,117.0


# Fix data types

In [42]:
cleaner.HP  = cleaner.HP.replace({"nan": np.nan})
cleaner.Atk = cleaner.Atk.replace({"nan": np.nan})
cleaner.Def = cleaner.Def.replace({"nan": np.nan})
cleaner.SpA = cleaner.SpA.replace({"nan": np.nan})
cleaner.SpD = cleaner.SpD.replace({"nan": np.nan})
cleaner.Spe = cleaner.Spe.replace({"nan": np.nan})

In [44]:
cleaner['HP']             = pd.to_numeric(cleaner['HP'])
cleaner['Atk']            = pd.to_numeric(cleaner['Atk'])
cleaner['Def']            = pd.to_numeric(cleaner['Def'])
cleaner['SpA']            = pd.to_numeric(cleaner['SpA'])
cleaner['SpD']            = pd.to_numeric(cleaner['SpD'])
cleaner['Spe']            = pd.to_numeric(cleaner['Spe'])
cleaner['height_m']       = pd.to_numeric(cleaner['height_m'])
cleaner['weight_kg']      = pd.to_numeric(cleaner['weight_kg'])
cleaner['base_happiness'] = pd.to_numeric(cleaner['base_happiness'])
cleaner['unique_id']      = cleaner['unique_id'].astype(np.int64)
#cleaner['is_legendary']   = cleaner['is_legendary'].astype(np.int64)
cleaner['abilities']      = cleaner['abilities'].astype(np.int64)

In [45]:
cleaner.dtypes

abilities           int64
base_happiness    float64
height_m          float64
is_legendary       object
poke_stats         object
rowid               int64
types              object
unique_id           int64
weight_kg         float64
rock                int64
dragon              int64
flying              int64
normal              int64
psychic             int64
fighting            int64
dark                int64
steel               int64
water               int64
bug                 int64
fairy               int64
ground              int64
poison              int64
fire                int64
grass               int64
ghost               int64
ice                 int64
nan                 int64
electric            int64
HP                float64
Atk               float64
Def               float64
SpA               float64
SpD               float64
Spe               float64
dtype: object

## Cleaning mostly done

In [46]:
cleaner.head()

,abilities,base_happiness,height_m,is_legendary,poke_stats,rowid,types,unique_id,weight_kg,rock,...,ghost,ice,nan,electric,HP,Atk,Def,SpA,SpD,Spe
0,3,70.0,0.6,0,"nan, 80.0, 55.0, 25.0, 35.0, 35.0",1,"fighting, nan",665,12.5,0,...,0,0,1,0,NaN,80.0,55.0,25.0,35.0,35.0
1,3,70.0,0.6,0,"nan, nan, 65.0, 45.0, 45.0, 45.0",2,"dragon, nan",288,29.7,0,...,0,0,1,0,NaN,NaN,65.0,45.0,45.0,45.0
2,2,70.0,1.2,0,"nan, 64.0, 60.0, 111.0, 60.0, 117.0",3,"poison, fire",349,22.2,0,...,0,0,0,0,NaN,64.0,60.0,111.0,60.0,117.0
3,3,70.0,0.4,0,"nan, 50.0, 95.0, 40.0, 50.0, 35.0",4,"ground, nan",260,6.5,0,...,0,0,1,0,NaN,50.0,95.0,40.0,50.0,35.0
4,2,70.0,0.6,0,"nan, 65.0, 64.0, 44.0, 48.0, 43.0",5,"water, nan",536,9.5,0,...,0,0,1,0,NaN,65.0,64.0,44.0,48.0,43.0


In [54]:
train.shape,test.shape,cleaner.shape

((520, 9), (520, 8), (1040, 34))

#### Split into train and test data

In [63]:
cleantrain = cleaner.iloc[0:520,:]
cleantest = cleaner.iloc[520:,:]
cleantrain['is_legendary'] = cleantrain['is_legendary'].astype(np.int64)
cleantrain.shape, cleantest.shape

C:\Users\Loopy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


((520, 34), (520, 34))

### Dropping unnecessary colums

In [64]:
cleantrain = cleantrain.drop(columns=['poke_stats'])
cleantrain = cleantrain.drop(columns=['types'])
cleantrain = cleantrain.drop(columns=['rowid'])
cleantrain = cleantrain.drop(columns=['unique_id'])

cleantest = cleantest.drop(columns=['rowid'])
cleantest = cleantest.drop(columns=['unique_id'])
cleantest = cleantest.drop(columns=['is_legendary'])
cleantest = cleantest.drop(columns=['poke_stats'])
cleantest = cleantest.drop(columns=['types'])
cleantrain.shape, cleantest.shape

((520, 30), (520, 29))

In [72]:
print(cleantrain.dtypes,"\n\n", cleantest.dtypes,end="")

abilities           int64
base_happiness    float64
height_m          float64
is_legendary        int64
weight_kg         float64
rock                int64
dragon              int64
flying              int64
normal              int64
psychic             int64
fighting            int64
dark                int64
steel               int64
water               int64
bug                 int64
fairy               int64
ground              int64
poison              int64
fire                int64
grass               int64
ghost               int64
ice                 int64
nan                 int64
electric            int64
HP                float64
Atk               float64
Def               float64
SpA               float64
SpD               float64
Spe               float64
dtype: object 

 abilities           int64
base_happiness    float64
height_m          float64
weight_kg         float64
rock                int64
dragon              int64
flying              int64
normal              i

In [73]:
cleantrain.to_csv('clean_train.csv', index=False)
cleantest.to_csv('clean_test.csv', index=False)